In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
CURR_DIR = os.getcwd()
BASE_DIR_PRICE = os.path.dirname(os.path.dirname(CURR_DIR))
BASE_DIR_SENTIMENT =  os.path.dirname(CURR_DIR)
DATA_DIR_PRICE = os.path.join(BASE_DIR_PRICE, 'Datasets\Price')
DATA_DIR_SENTIMENT = os.path.join(BASE_DIR_SENTIMENT, '02 Sentiment Analysis\Datasets')

In [3]:
# uncomment print statements for details
def merge_frames(df_price, df_sentiment, df_additional_price_data=None):
    
    # Column Names lower case
    df_price.columns = df_price.columns.str.lower()
    # Convert date column to datetime object
    df_price.date = pd.to_datetime(df_price.date, infer_datetime_format=True)
    
    if df_additional_price_data is not None:
             # Column Names lower case
            df_additional_price_data.columns = df_additional_price_data.columns.str.lower()
            # Convert date column to datetime object
            df_additional_price_data.date = pd.to_datetime(df_additional_price_data.date, infer_datetime_format=True)
            # Merge price dfs
            df_price = btc_price.merge(df_additional_price_data, on='date', how='inner')
    
#     print('Price data:\n',min(df_price.date),'-',max(df_price.date),'\n')
#     print(df_price.isna().any(),'\n')
#     print(df_price.isna().sum(),'\n')
#     print('Columns:',df_price.shape[1])
#     print('Rows:',df_price.shape[0],'\n')
    
     # Create col for number of articles
    df_sentiment['num_art'] = 1
    # Group sentiment df on date
    df_sentiment = df_sentiment.groupby('date').agg({'textBlob_polarity':'mean',
                                   'textBlob_subjectivity':'mean',
                                   'vader_compound':'mean',
                                   'vader_negative_score':'mean',
                                   'vader_neutral_score':'mean',
                                   'vader_positive_score':'mean',
                                   'finBert_text_positive_score':'mean',
                                   'finBert_text_negative_score':'mean',
                                   'finBert_text_neutral_score':'mean',
                                   'finBert_title_positive_score':'mean',
                                   'finBert_title_negative_score':'mean',
                                   'finBert_title_neutral_score':'mean',
                                   'num_art':'count'})
    
    # Reset index and convert date column to datetime object
    df_sentiment = df_sentiment.reset_index()
    df_sentiment.date = pd.to_datetime(df_sentiment.date, infer_datetime_format=True)
    
#     print('Sentiment data:\n', min(df_sentiment.date),'-',max(df_sentiment.date),'\n')
#     print(df_sentiment.isna().any(),'\n')
#     print(df_sentiment.isna().sum(),'\n')
#     print('Columns:',df_sentiment.shape[1])
#     print('Rows:',df_sentiment.shape[0],'\n')
    
    
    # Merge 
    df = pd.merge(df_price, df_sentiment, on='date', how='left')
    df.set_index('date', inplace=True)
    
#     print('Merged data:\n', df.iloc[-1].name,'-',df.iloc[0].name,'\n')
#     print(df.isna().any(),'\n')
#     print(df.isna().sum(),'\n')
#     print(df.isna().sum().sum(),'datapoints were interpolated','\n')
#     print('Columns:',df.shape[1])
#     print('Rows:',df.shape[0],'\n')
    
    
    # Interpolate data
    df = df.interpolate(method='time', limit_direction='both')
    
    # Create categorical sentiment data
    df['finbert_text_label'] = df.apply(get_finbert_text_sentiment,  axis=1)
    df['finbert_title_label'] = df.apply(get_finbert_title_sentiment,  axis=1)
    df['textBlob_label'] = df.textBlob_polarity.apply(get_textBlob_sentiment)
    df['vader_label'] = df.vader_compound.apply(get_vader_sentiment)
    df['lm_label'] = df.vader_compound.apply(get_lm_sentiment)
    df['finbert_score'] = df[['finBert_title_positive_score', 'finBert_title_negative_score', 'finBert_title_neutral_score']].max(axis=1)
    
    # Drop na
    print('Final data:\n', df.iloc[-1].name,'-',df.iloc[0].name,'\n')
    print(df_sentiment.isna().any(),'\n')
    print(df_sentiment.isna().sum(),'\n')
    print('Columns:',df.shape[1])
    print('Rows:',df.shape[0],'\n')
    df = df.dropna()
    df.reset_index(inplace=True)
    return df
    

In [4]:
# Functions for sentiment dfs
def get_textBlob_sentiment(score):
    if score > 0:
        return 'positive'
    elif score == 0:
        return 'neutral'
    else:
        return 'negative'

def get_vader_sentiment(score):
    if score >= 0.5:
        return 'positive'
    elif score <= -0.5:
        return 'negative'
    elif score > -0.5 and score < 0.5:
        return 'neutral'

def get_finbert_text_sentiment(df):
    scores = df[['finBert_text_positive_score', 'finBert_text_negative_score', 'finBert_text_neutral_score']].values
    sentiment = ['positive', 'negative', 'neutral']
    return sentiment[np.argmax(scores)]

def get_finbert_title_sentiment(df):
    scores = df[['finBert_title_positive_score', 'finBert_title_negative_score', 'finBert_title_neutral_score']].values
    sentiment = ['positive', 'negative', 'neutral']
    return sentiment[np.argmax(scores)]

def get_lm_sentiment(score):
    if score == 0:
        return 'neutral'
    elif score >0:
        return 'positive'
    elif score <0:
        return 'negative'

In [5]:
# Import price data
filenames = ['binance-coin_price_data_all_time.csv', 'bitcoin_price_data_all_time.csv', 
             'cardano_price_data_all_time.csv','ethereum_price_data_all_time.csv',
            'xrp_price_data_all_time.csv', 'bitcoin_price_data_additional.csv']
dfs_price ={}
for filename in filenames:
    filepath = os.path.join(DATA_DIR_PRICE, filename)
    dfs_price[filename] = pd.read_csv(filepath)

In [6]:
# Import sentiment data
filenames = ['btc_all_sentiment.csv', 'eth_all_sentiment.csv', 'xrp_all_sentiment.csv','ada_all_sentiment.csv','bnb_all_sentiment.csv']

#filenames = ['title_btc_all_sentiment.csv','text_btc_all_sentiment.csv','btc_sentiment.csv', 'eth_all_sentiment.csv', 'xrp_all_sentiment.csv','ada_all_sentiment.csv','bnb_all_sentiment.csv']

dfs_sentiment={}
for filename in filenames:
    filepath = os.path.join(DATA_DIR_SENTIMENT, filename)
    dfs_sentiment[filename] = pd.read_csv(filepath)

In [7]:
btc_price= dfs_price['bitcoin_price_data_all_time.csv']
btc_ad = dfs_price['bitcoin_price_data_additional.csv']
btc_sentiment = dfs_sentiment['btc_all_sentiment.csv']

In [8]:
btc = merge_frames(btc_price,btc_sentiment, btc_ad)

Price data:
 2013-04-28 00:00:00 - 2022-12-29 00:00:00 

date            False
open            False
high            False
low             False
close           False
volume          False
market cap      False
bchain/diff     False
bchain/avbls    False
bchain/hrate    False
bchain/totbc    False
dtype: bool 

date            0
open            0
high            0
low             0
close           0
volume          0
market cap      0
bchain/diff     0
bchain/avbls    0
bchain/hrate    0
bchain/totbc    0
dtype: int64 

Columns: 11
Rows: 3533 

Sentiment data:
 2013-04-01 00:00:00 - 2022-12-30 00:00:00 

date                            False
textBlob_polarity               False
textBlob_subjectivity           False
vader_compound                  False
vader_negative_score            False
vader_neutral_score             False
vader_positive_score            False
finBert_text_positive_score     False
finBert_text_negative_score     False
finBert_text_neutral_score      False
finBert_

In [9]:
btc.to_csv( os.path.join(CURR_DIR, 'Datasets', 'btc.csv'), index=False)

In [10]:
eth_price= dfs_price['ethereum_price_data_all_time.csv']
eth_sentiment = dfs_sentiment['eth_all_sentiment.csv']

In [11]:
eth = merge_frames(eth_price,eth_sentiment)

Price data:
 2015-08-07 00:00:00 - 2022-12-30 00:00:00 

date          False
open          False
high          False
low           False
close         False
volume        False
market cap    False
dtype: bool 

date          0
open          0
high          0
low           0
close         0
volume        0
market cap    0
dtype: int64 

Columns: 7
Rows: 2703 

Sentiment data:
 2015-10-06 00:00:00 - 2022-12-28 00:00:00 

date                            False
textBlob_polarity               False
textBlob_subjectivity           False
vader_compound                  False
vader_negative_score            False
vader_neutral_score             False
vader_positive_score            False
finBert_text_positive_score     False
finBert_text_negative_score     False
finBert_text_neutral_score      False
finBert_title_positive_score    False
finBert_title_negative_score    False
finBert_title_neutral_score     False
num_art                         False
dtype: bool 

date                           

In [12]:
xrp_price= dfs_price['xrp_price_data_all_time.csv']
xrp_sentiment = dfs_sentiment['xrp_all_sentiment.csv']

In [13]:
xrp = merge_frames(xrp_price,xrp_sentiment)

Price data:
 2013-08-04 00:00:00 - 2022-12-30 00:00:00 

date          False
open          False
high          False
low           False
close         False
volume        False
market cap    False
dtype: bool 

date          0
open          0
high          0
low           0
close         0
volume        0
market cap    0
dtype: int64 

Columns: 7
Rows: 3436 

Sentiment data:
 2013-06-05 00:00:00 - 2022-12-28 00:00:00 

date                            False
textBlob_polarity               False
textBlob_subjectivity           False
vader_compound                  False
vader_negative_score            False
vader_neutral_score             False
vader_positive_score            False
finBert_text_positive_score     False
finBert_text_negative_score     False
finBert_text_neutral_score      False
finBert_title_positive_score    False
finBert_title_negative_score    False
finBert_title_neutral_score     False
num_art                         False
dtype: bool 

date                           

In [14]:
ada_price= dfs_price['cardano_price_data_all_time.csv']
ada_sentiment = dfs_sentiment['ada_all_sentiment.csv']

In [15]:
ada = merge_frames(ada_price,ada_sentiment)

Price data:
 2017-10-01 00:00:00 - 2022-12-30 00:00:00 

date          False
open          False
high          False
low           False
close         False
volume        False
market cap    False
dtype: bool 

date          0
open          0
high          0
low           0
close         0
volume        0
market cap    0
dtype: int64 

Columns: 7
Rows: 1917 

Sentiment data:
 2017-01-03 00:00:00 - 2022-12-28 00:00:00 

date                            False
textBlob_polarity               False
textBlob_subjectivity           False
vader_compound                  False
vader_negative_score            False
vader_neutral_score             False
vader_positive_score            False
finBert_text_positive_score     False
finBert_text_negative_score     False
finBert_text_neutral_score      False
finBert_title_positive_score    False
finBert_title_negative_score    False
finBert_title_neutral_score     False
num_art                         False
dtype: bool 

date                           

In [16]:
bnb_price= dfs_price['binance-coin_price_data_all_time.csv']
bnb_sentiment = dfs_sentiment['bnb_all_sentiment.csv']

In [17]:
bnb = merge_frames(bnb_price,bnb_sentiment)

Price data:
 2017-07-25 00:00:00 - 2022-12-30 00:00:00 

date          False
open          False
high          False
low           False
close         False
volume        False
market cap    False
dtype: bool 

date          0
open          0
high          0
low           0
close         0
volume        0
market cap    0
dtype: int64 

Columns: 7
Rows: 1985 

Sentiment data:
 2018-04-15 00:00:00 - 2022-12-26 00:00:00 

date                            False
textBlob_polarity               False
textBlob_subjectivity           False
vader_compound                  False
vader_negative_score            False
vader_neutral_score             False
vader_positive_score            False
finBert_text_positive_score     False
finBert_text_negative_score     False
finBert_text_neutral_score      False
finBert_title_positive_score    False
finBert_title_negative_score    False
finBert_title_neutral_score     False
num_art                         False
dtype: bool 

date                           

In [18]:
dataframes = [btc, xrp, eth, ada, bnb]
filenames = ['btc.csv','xrp.csv', 'eth.csv', 'ada.csv', 'bnb.csv']

for df, filename in zip(dataframes, filenames):
    filepath = os.path.join(CURR_DIR, 'Datasets', filename)
    df.to_csv(filepath, index=False)